In [1]:
%matplotlib inline
from ggplot import *
import matplotlib as mpl
import seaborn as sns
mpl.rcParams['figure.figsize'] = (15, 6)
mpl.style.use('ggplot')
import pandas as pd
import numpy
import subprocess
import glob
import re
import os, sys
from collections import defaultdict, Counter, OrderedDict
from Bio import SeqIO, SeqRecord, Seq
from tRNA_position import *
pd.set_option('display.max_colwidth',10000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows',1000)
isotypes = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Glu', 'Gly', 'His', 'Ile', 'iMet', 'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val']

/projects/lowelab/users/blin/anaconda3/lib/python3.5/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/projects/lowelab/users/blin/anaconda3/lib/python3.5/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/projects/lowelab/users/blin/anaconda3/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Introduction

A global view of identity elements versus biological features would be a powerful tool for predicting tRNA function using primary sequence. To do this, I'll need to align eukaryotic tRNAs to our established models and annotate positions based on universal numbering. Then, I'll extract sequence information such as position, clade, or variable arm length.

# Process tRNAs
## Species information

In [2]:
species_table = pd.read_table('genomes-021618.tsv', header=None, names=['species', 'longname', 'domain', 'clade', 'taxid'], dtype={'taxid': str})
species_table.head()

,species,longname,domain,clade,taxid
0,ashbGoss_ATCC10895,Eremothecium gossypii ATCC 10895,eukaryota,Fungi,33169
1,aspeFumi_AF293,Aspergillus fumigatus Af293,eukaryota,Fungi,746128
2,aspeNidu_FGSC_A4,Aspergillus nidulans FGSC A4,eukaryota,Fungi,162425
3,aspeOryz_RIB40,Aspergillus oryzae RIB40,eukaryota,Fungi,5062
4,botrCine_B05_10,Botrytis cinerea B05.10,eukaryota,Fungi,40559


## Align tRNAs to Sprinzl numbering model

We have a "quality set" of tRNAs, which include tRNAs from previous tRNAscan-SE runs and newer runs. We also want to keep track of high-scoring, isotype mismatched tRNAs.

In [3]:
species = sorted(glob.glob("out/*-detailed.out"))
species = [sp[4:-19] for sp in species]
seqs = []
for sp in species:
  sys.stdout.write('processing ' + sp + '...')
  sys.stdout.flush()
  
  tRNA_file = 'tRNAs/' + sp + '-tRNAs.fa'
  tscanout_file = 'out/{}-tRNAs-detailed.out'.format(sp)
  # Import annotation from tRNAscan .out files. We want to keep the ones that are "Quality set" or "Unexpected anticodon;First-pass quality filtered".
  approved_tRNAs = []
  intron_lengths = [] # we are purging introns to improve alignment, so store intron lengths here
  for metadata in pd.read_table(tscanout_file, sep="\t", skiprows=3, header=None).iterrows():
    if (metadata[1].iloc[-1] in ["Quality set", "Unexpected anticodon;First-pass quality filtered", "quality set", "unexpected anticodon,first-pass quality filtered", "Isotype mismatch;First-pass quality filtered", "First-pass quality filtered", "first-pass quality filtered"]
        or re.match('^IPD:-\d+\.\d+,isotype mismatch,first-pass quality filtered$', metadata[1].iloc[-1])):
      approved_tRNAs.append('{}.trna{}-{}{}'.format(metadata[1][0], metadata[1][1], metadata[1][4], metadata[1][5]))
      intron_length = abs(int(metadata[1][6]) - int(metadata[1][7]))
      if intron_length > 0: intron_length = intron_length + 1
      intron_lengths.append(intron_length)
  # Remove introns using sstofa
  subprocess.call('sstofa3 ss/{}-tRNAs.ss "" 1 0 > {}'.format(sp, tRNA_file), shell=True)

  # Parse isotype-specific scores file
  iso_scores = pd.read_table('iso/' + sp + '-tRNAs.iso', header=0)
  iso_scores['Undet'] = 0
  iso_scores['Sup'] = 0
  iso_scores['best'] = iso_scores.iloc[:,2:].idxmax(axis=1)
  iso_scores['score'] = iso_scores.max(axis=1, numeric_only=True)
  iso_scores['ac_score'] = iso_scores.lookup(iso_scores.index, iso_scores.iloc[:, 1])
  iso_scores.index = iso_scores.tRNAscanID.values
  iso_scores = iso_scores[['best', 'score', 'ac_score']]
  
  # Parse seqs, filtering out the ones we don't need, and add to proper isotype
  for seq in SeqIO.parse('tRNAs/{}-tRNAs.fa'.format(sp), 'fasta'):
    if seq.id not in approved_tRNAs: continue
    if "pseudogene" in seq.description: continue
    intron_length = intron_lengths[approved_tRNAs.index(seq.id)]
    score = float(re.findall('Sc: [\d\.]+', seq.description)[0].split()[-1])
    # relax tRNA score requirements for fungi
    if sp in species_table.species.values and species_table[species_table.species == sp].clade.values[0] == "Fungi":
      if score < 30 and seq.description != 'chrI.trna9-MetCAT (3515186-3515314)  Met (CAT) 70 bp  Sc: 27.6': continue
    else:
      if score < 50: continue
    trnascanid = re.findall('.+\.trna\d+', seq.id)[0]
    best_isotype = iso_scores.loc[trnascanid].best
    if best_isotype == 'SeC' or 'mito' in best_isotype or 'mt' in trnascanid: continue
    isoscore, ac_score = iso_scores.loc[trnascanid].score, iso_scores.loc[trnascanid].ac_score
    seq.id = '{}|{} Iso: {} ({}) Iso_ac: {} Intron: {}'.format(sp, seq.description, isoscore, best_isotype, ac_score, intron_length)
    seq.description = ''
    seqs.append(seq)
    
  print('finished')
  sys.stdout.flush()

processing ailMel1...finished
processing anoCar2...finished
processing anoGam2...finished
processing apiMel1...finished
processing araTha1...finished
processing ashbGoss_ATCC10895...finished
processing aspeFumi_AF293...finished
processing aspeNidu_FGSC_A4...finished
processing aspeOryz_RIB40...finished
processing balAcu1...finished
processing bomTer1...finished
processing bosTau8...finished
processing botrCine_B05_10...finished
processing braDis3...finished
processing braOle1...finished
processing bruMal1...finished
processing caeJap1...finished
processing caePb2...finished
processing caeRem3...finished
processing calJac3...finished
processing calMil1...finished
processing canFam3...finished
processing candAlbi_WO_1...finished
processing candDubl_CD36...finished
processing candGlab_CBS_138...finished
processing candOrth_CO_90_125...finished
processing carPap1...finished
processing cavPor3...finished
processing cb3...finished
processing ce11...finished
processing cerSim1...finished
proc

In [4]:
fasta_handle = open('euk-tRNAs.fa', 'w')
SeqIO.write(seqs, fasta_handle, 'fasta')
fasta_handle.close()
num_model = '/projects/lowelab/users/blin/tRNAscan/models/domain-specific/euk-num-092016.cm'
subprocess.call('cmalign -g --notrunc -o euk-tRNAs.sto {} euk-tRNAs.fa'.format(num_model), shell=True)

0

## Create table of tRNA bases by position

This is a giant data frame with one row per tRNA, and with columns for each position, plus tRNA metadata like species and loop lengths.

In [5]:
def position_base(positions, seq):
  for position_index, position in enumerate(positions):
    if position.paired:
      index1, index2 = position.position.split(':')
      index1, index2 = int(index1), int(index2)
      base_pair = "{}:{}".format(seq[index1 - 1].upper(), seq[index2 - 1].upper())
      yield position.sprinzl, base_pair
    else:
      index = int(position.position)
      base = seq[index - 1].upper()
      yield position.sprinzl, base
      
identities = pd.DataFrame()

# get positions
alignment_fhandle = open('euk-tRNAs.sto')
positions = [] # list containing each position in the tRNA

# first, get secondary structure
for line in alignment_fhandle:
  if line[0:12] == '#=GC SS_cons':
    ss = line.strip().split()[-1]
alignment_fhandle.close()

# parse secondary structure into regions and positions
positions = annotate_positions(ss)

# get nucleotide at each position for each tRNA by parsing Stockholm file
alignment_fhandle = open('euk-tRNAs.sto')
trnas = []
skipped = []
for line in alignment_fhandle:
  if line[0] in ["#", '\n', '/']: continue
  species, desc = line.strip().split('|', 1)
  seqname = desc.split()[0]
  seq = desc.split()[-1]
  if any(species_table.species == species):
    row = species_table[species_table.species == species]
  else: 
    skipped.append(species)
    continue
  isotype = re.findall('\.trna\d+-([A-Za-z]+)', seqname)[0][:-3]
  seqname = '{}_{}'.format(species, seqname)
  trna = {'domain': row.domain.values[0], 'clade': row.clade.values[0], 'species': species, 'species_long': row.longname.values[0], 'taxid': row.taxid.values[0], 'seqname': seqname, 'isotype': isotype}
  trna = {**trna, **{sprinzl: base for sprinzl, base in position_base(positions, seq)}}
  trnas.append(trna)

if len(skipped) > 0: print('skipped the following: {}'.format(set(skipped)))
identities = identities.append(trnas, ignore_index=True)
identities.fillna('.', inplace=True)
alignment_fhandle.close()

skipped the following: {'saccCere_VIN13', 'saccCere_R008', 'danRer10', 'saccCere_P301', 'saccCere_VL3', 'saccCere_UFMG_A_905', 'saccCere_SIGMA1278B', 'saccCere_FOSTERSB', 'saccCere_R103', 'saccCere_AWRI796', 'saccCere_YJSH1', 'saccCere_CEN_PK113_7D', 'saccCere_P283', 'saccCere_W303'}


### Create single base columns from paired positions

This is mainly for plotting and such.

In [6]:
cols = list(filter(lambda x: ':' in x, identities.columns))
for col in cols:
  pos1, pos2 = col.split(':')
  base1 = [bases.split(':')[0] for bases in identities[col]]
  base2 = [bases.split(':')[1] for bases in identities[col]]
  identities[pos1] = base1
  identities[pos2] = base2

### Annotate tertiary interactions

We also want to annotate the 2-base version of the base triples. This makes checking for conserved interactions (e.g. an identity element) a bit easier. It's important to remember to consider the 3rd base pair too, though.

In [7]:
identities['8:14:21'] = identities.loc[:, ["8", "14", "21"]].apply(lambda row: ':'.join(row), axis=1)
identities['8:14'] = identities.loc[:, ["8", "14"]].apply(lambda row: ':'.join(row), axis=1)
identities['9:12:23'] = identities.loc[:, ["9", "12", "23"]].apply(lambda row: ':'.join(row), axis=1)
identities['9:23'] = identities.loc[:, ["9", "23"]].apply(lambda row: ':'.join(row), axis=1)
identities['10:25:45'] = identities.loc[:, ["10", "25", "45"]].apply(lambda row: ':'.join(row), axis=1)
identities['10:45'] = identities.loc[:, ["10", "45"]].apply(lambda row: ':'.join(row), axis=1)
identities['13:22:46'] = identities.loc[:, ["13", "22", "46"]].apply(lambda row: ':'.join(row), axis=1)
identities['22:46'] = identities.loc[:, ["22", "46"]].apply(lambda row: ':'.join(row), axis=1)
identities['15:48'] = identities.loc[:, ["15", "48"]].apply(lambda row: ':'.join(row), axis=1)
identities['18:55'] = identities.loc[:, ["18", "55"]].apply(lambda row: ':'.join(row), axis=1)
identities['19:56'] = identities.loc[:, ["19", "56"]].apply(lambda row: ':'.join(row), axis=1)
identities['26:44'] = identities.loc[:, ["26", "44"]].apply(lambda row: ':'.join(row), axis=1)
identities['54:58'] = identities.loc[:, ["54", "58"]].apply(lambda row: ':'.join(row), axis=1)

### Additional sequence information

In [8]:
# Basic information - isotype, anticodon, score
seqinfo = []
for line in open('euk-tRNAs.sto'):
  if line[0:4] != "#=GS": continue
  _, desc, _, _, isotype, anticodon, _, _, _, score, _, isoscore, isotype_best, _, isoscore_ac, _, intron_length = line.strip().split()
  seqname = desc.replace('|', '_', 1)
  seqinfo.append([seqname, isotype_best[1:-1], anticodon[1:-1], float(score), float(isoscore), float(isoscore_ac), int(intron_length)])
seqinfo = pd.DataFrame(seqinfo, columns=['seqname', 'isotype_best', 'anticodon', 'score', 'isoscore', 'isoscore_ac', 'intron'])
identities = identities.merge(seqinfo, on='seqname')

# GC content
paired_cols = identities.columns[list(map(lambda x: (':' in x), identities.columns))]
identities['GC'] = identities[paired_cols].apply(lambda x: sum((x == "G:C") | (x == "C:G"))/len(paired_cols), axis=1)

### Adjust isotype designations

In [10]:
# iMet may or may not be properly annotated, depending on tRNAscan-SE run
unlifted_imets = identities[(identities.isotype == "Met") & (identities.isotype_best == "iMet")].index
identities.loc[unlifted_imets, 'isotype'] = 'iMet'

# Manage tRNAs of missing isotypes
for sp in identities.species.unique():
  # Lift the highest-scoring iMet if species does not contain iMet
  if 'iMet' not in identities[identities.species == sp].isotype.values:
    # reparse iso file and re-assign the Met with the highest iMet score
    imet_scores = {}
    seqname_isotypes = {}
    for line in open('iso/{}-tRNAs.iso'.format(sp)):
      tabs = line.strip().split('\t')
      if tabs[0] == 'tRNAscanID': continue
      seqname = sp + '_' + tabs[0]
      imet_scores[seqname] = float(tabs[-1])
      seqname_isotypes[seqname] = tabs[1]
    
    best_seqname = max(imet_scores, key = imet_scores.get)
    best_score = numpy.max(imet_scores.values())
    identities.loc[identities.seqname.apply(lambda x: x[:-7]) == best_seqname, 'isotype'] = 'iMet'
    for seqname in imet_scores:
      if abs(imet_scores[seqname] - imet_scores[best_seqname]) < 2 and seqname_isotypes[seqname] == seqname_isotypes[best_seqname]:
        identities.loc[identities.seqname.apply(lambda x: x[:-7]) == seqname, 'isotype'] = 'iMet'
    
  # Revert tRNAs in species that are missing tRNAs of an isotype
  for isotype in isotypes:
    if isotype not in identities.loc[identities.species == sp].isotype.values:
      print('addressing', sp, isotype)
      identities.loc[(identities.species == sp) & (identities.seqname.apply(lambda x: isotype in x)), 'isotype'] = isotype

addressing chaeTher_VAR_THERMOPHILUM_DSM1 Tyr
addressing crypGatt_WM276 Asp
addressing crypNeof_VAR_GRUBII_H99 Asp
addressing crypNeof_VAR_NEOFORMANS_B_3501 Asp
addressing crypNeof_VAR_NEOFORMANS_JEC21 Asp


### Insertions/deletions

In [11]:
# Insertions (minus misaligned introns at 37/38)
intron_cols = list(filter(lambda x: x[0:3] == '37i', identities.columns))
insertion_cols = list(filter(lambda x: bool(re.search('^\d+i', x)) & (x not in intron_cols), identities.columns))
identities['insertions'] = identities[insertion_cols].apply(lambda x: sum(x != '.'), axis=1)

# Deletions at positions that are not the variable arm, and not counting 17/17a/20a/20b
base_cols = list(filter(lambda x: bool(re.match('^\d+$', x)) & (x not in ['74', '75', '76', '17', '17a', '20a', '20b']), identities.columns))
identities['deletions'] = identities[base_cols].apply(lambda x: ''.join(x).count('-'), axis=1)

### Loop sizes

In [12]:
def bounds_to_cols(cols, start, end):
  selected_cols = []
  for col in cols:
    matches = re.findall('\d+', col)
    if len(matches) < 1: continue
    index = int(matches[0])
    if (index >= start and index <= end or col[0:3] == '{}i'.format(start - 1)) and col[0] != 'V':
      selected_cols.append(col)
  return selected_cols

dloop_cols = list(filter(lambda col: ':' not in col, bounds_to_cols(identities.columns, 14, 21)))
identities['D-loop'] = identities[dloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

# Leu, Ser have a 3 bp D stem
dloop_II_cols = list(filter(lambda col: ':' not in col, bounds_to_cols(identities.columns, 13, 22)))
identities.ix[(identities.isotype == 'Leu') | (identities.isotype == 'Ser'), 'D-loop'] = identities[dloop_II_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

acloop_cols = list(filter(lambda x: not re.match('37i.+', x), bounds_to_cols(identities.columns, 32, 38)))
identities['AC-loop'] = identities[acloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

tpcloop_cols = bounds_to_cols(identities.columns, 54, 60)
identities['TPC-loop'] = identities[tpcloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

varm_cols = list(filter(lambda x: ('V' in x) & (':' not in x) | (x in bounds_to_cols(identities.columns, 45, 48)), identities.columns))
identities['V-arm'] = identities[varm_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

/projects/lowelab/users/blin/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  app.launch_new_instance()


### High-quality tRNA set

There was talk about using the high quality tRNAs to look at identity elements. Instead, I will just annotate specific tRNAs as high-quality or not. We can sort out the differences later.

In [13]:
# Import list of quality tRNAs
quality_trnas = [line.strip() for line in open('quality-set.out')]
identities['quality'] = identities.seqname.isin(quality_trnas)

### Restrict tRNAs by species

We may also want to limit the contribution of any single species, similar to how we built the isotype-specific models. The restricted set is a subset of the quality set.

In [14]:
identities.ix[:, 'restrict'] = False
for species in identities.species.unique():
  species_df = identities.ix[identities.species == species, :]
  for isotype in isotypes:
    isotype_indices = identities.ix[(identities.species == species) & (identities.isotype == isotype) & (-identities.restrict), ].index
    quality_indices = identities.ix[(identities.species == species) & (identities.isotype == isotype) & (-identities.restrict) & (identities.quality), ].index
    unique_scores_indices = isotype_indices[-identities.ix[isotype_indices, 'score'].duplicated()]
    unique_quality_indices = quality_indices[-identities.ix[quality_indices, 'score'].duplicated()]
    
    if len(unique_quality_indices) >= 50:
      unique_quality_indices = unique_quality_indices[numpy.argsort(identities.ix[unique_quality_indices, 'score'])][::-1][:50]
      restricted_indices = list(set(isotype_indices) - set(unique_quality_indices))
    elif len(unique_quality_indices) > 0:
      if len(unique_scores_indices) > 50 - len(unique_quality_indices):
        unique_scores_indices = unique_scores_indices[numpy.argsort(identities.ix[unique_scores_indices, 'score'])][::-1][:50 - len(unique_quality_indices)]
      restricted_indices = list(set(isotype_indices) - set(unique_quality_indices) - set(unique_scores_indices))
    else:
      if len(unique_scores_indices) > 50:
        unique_scores_indices = unique_scores_indices[numpy.argsort(identities.ix[unique_scores_indices, 'score'])][::-1][:50]
      restricted_indices = list(set(isotype_indices) - set(unique_scores_indices))
    
    identities.ix[restricted_indices, 'restrict'] = True

The quality set does have some issues, though: it filters out rare codons, when rare is relative to model species. It also performs a hard check for anticodon-isotype model match. Fungi have much more diverged tRNAs and, in many cases, a different isotype scores higher even when it's clear that it's the bona fide tRNA (mostly based on number of tRNAs of that isotype, or codon usage). 

## Export to R

R has superior visualization capabilities.

### Order columns

To make it look pretty.

In [15]:
def position_str_to_int(position):
  if position == "20a": return 20.1
  if position == "20b": return 20.2
  digits = re.findall('\d+', position)
  if len(digits) == 0: return -1
  insert = 0
  if 'i' in position and len(digits) == 2: insert = float(digits[1]) / 1000
  if position[0] == 'V':
    if ':' in position: return int(digits[0]) + 45 - 10 + insert # V11~V17
    else: return int(digits[0]) + 45 + 7 + insert # V1~V5
  if int(digits[0]) >= 46: return int(digits[0]) + 50 + insert # just add an arbitrarily large number to skip v-arm
  return int(digits[0]) + insert

identities = identities[sorted(list(identities.columns), key=position_str_to_int)]

In [16]:
identities.to_csv(path_or_buf='identities.tsv', sep='\t', index_label=False, index=False)

In [17]:
# Export fasta of tRNAs used for analysis
fasta_handle = open('euk-tRNAs-filtered.fa', 'w')
filtered_seqs = []
seqnames = identities[~identities.restrict].seqname.tolist()
for seq in SeqIO.parse('euk-tRNAs.fa', 'fasta'):
  seqname = seq.name.replace('|', '_')
  if seqname in seqnames: filtered_seqs.append(seq)
SeqIO.write(filtered_seqs, fasta_handle, 'fasta')
fasta_handle.close()

In [18]:
identities.head()

,clade,domain,isotype,seqname,species,species_long,taxid,isotype_best,anticodon,score,isoscore,isoscore_ac,intron,GC,insertions,deletions,D-loop,AC-loop,TPC-loop,V-arm,quality,restrict,1:72,1,1i1,2:71,2,2i1,3:70,3,3i1,3i2,3i3,3i4,3i5,3i6,4:69,4,4i1,4i2,4i3,4i4,4i5,4i6,4i7,4i8,4i9,4i10,4i11,5:68,5,5i1,5i2,5i3,5i4,5i5,5i6,5i7,6:67,6,6i1,7:66,7,7i1,7i2,7i3,7i4,7i5,7i6,7i7,7i8,7i9,7i10,7i11,7i12,7i13,7i14,8,8:14:21,8:14,8i1,8i2,8i3,8i4,9,9:12:23,9:23,9i1,9i2,9i3,10:25,10,10:25:45,10:45,10i1,10i2,11:24,11,12:23,12,12i1,13:22,13,13:22:46,14,14i1,14i2,14i3,14i4,14i5,14i6,14i7,14i8,14i9,14i10,14i11,14i12,14i13,14i14,14i15,14i16,14i17,14i18,14i19,14i20,14i21,14i22,14i23,14i24,14i25,14i26,14i27,14i28,14i29,14i30,14i31,14i32,14i33,14i34,14i35,14i36,14i37,14i38,14i39,14i40,14i41,14i42,14i43,14i44,14i45,14i46,14i47,14i48,15,15:48,16,16i1,16i2,16i3,16i4,16i5,16i6,16i7,16i8,16i9,16i10,16i11,16i12,16i13,16i14,16i15,16i16,17,17a,18,18:55,19,19:56,19i1,19i2,19i3,19i4,19i5,19i6,19i7,19i8,19i9,19i10,19i11,19i12,19i13,19i14,19i15,19i16,19i17,19i18,19i19,19i20,19i21,19i22,19i23,19i24,19i25,19i26,19i27,19i28,19i29,19i30,19i31,19i32,19i33,19i34,19i35,19i36,19i37,19i38,19i39,19i40,19i41,19i42,19i43,19i44,19i45,19i46,19i47,19i48,19i49,19i50,19i51,19i52,19i53,19i54,19i55,19i56,19i57,20,20i1,20i2,20i3,20i4,20i5,20i6,20a,20b,21,22,22:46,22i1,23,23i1,23i2,23i3,23i4,23i5,23i6,24,24i1,25,25i1,25i2,25i3,25i4,25i5,25i6,25i7,25i8,25i9,25i10,25i11,25i12,25i13,25i14,25i15,25i16,25i17,25i18,25i19,25i20,25i21,25i22,25i23,25i24,25i25,25i26,25i27,25i28,25i29,25i30,25i31,25i32,25i33,25i34,25i35,25i36,25i37,25i38,25i39,25i40,25i41,25i42,25i43,25i44,25i45,25i46,25i47,25i48,25i49,25i50,25i51,25i52,25i53,25i54,25i55,25i56,25i57,26,26:44,26i1,26i2,26i3,26i4,26i5,26i6,26i7,26i8,26i9,26i10,26i11,26i12,26i13,26i14,26i15,26i16,26i17,26i18,26i19,26i20,26i21,26i22,26i23,26i24,26i25,26i26,26i27,26i28,26i29,26i30,26i31,26i32,26i33,26i34,26i35,26i36,26i37,26i38,26i39,26i40,26i41,26i42,26i43,26i44,26i45,26i46,26i47,26i48,26i49,26i50,26i51,26i52,26i53,27:43,27,27i1,27i2,27i3,27i4,28:42,28,28i1,29:41,29,29i1,29i2,30:40,30,30i1,31:39,31,32,33,34,35,35i1,36,37,37i1,37i2,37i3,37i4,37i5,37i6,37i7,37i8,37i9,37i10,37i11,37i12,37i13,37i14,37i15,37i16,37i17,37i18,37i19,37i20,37i21,37i22,38,38i1,39,39i1,40,40i1,40i2,40i3,40i4,41,41i1,41i2,41i3,41i4,42,42i1,43,44,44i1,44i2,44i3,44i4,44i5,44i6,44i7,44i8,44i9,44i10,44i11,44i12,44i13,44i14,44i15,44i16,44i17,44i18,44i19,44i20,44i21,44i22,44i23,45,V11:V21,V12:V22,V13:V23,V14:V24,V15:V25,V16:V26,V17:V27,V1,V2,V3,V4,V5,V11,V12,V13,V14,V15,V16,V17,V21,V22,V23,V24,V25,V26,V27,46,47,47i1,47i2,47i3,47i4,48,49:65,49,49i1,50:64,50,50i1,50i2,50i3,50i4,50i5,50i6,50i7,50i8,51:63,51,51i1,51i2,51i3,51i4,52:62,52,52i1,53:61,53,53i1,53i2,54,54:58,54i1,54i2,54i3,54i4,54i5,55,55i1,55i2,55i3,55i4,55i5,55i6,55i7,55i8,55i9,55i10,55i11,55i12,56,56i1,56i2,56i3,56i4,56i5,56i6,57,57i1,58,58i1,58i2,58i3,58i4,58i5,58i6,58i7,58i8,58i9,58i10,58i11,58i12,58i13,58i14,58i15,58i16,58i17,59,59i1,60,60i1,60i2,60i3,60i4,60i5,60i6,60i7,60i8,60i9,60i10,60i11,60i12,60i13,60i14,61,61i1,62,63,64,64i1,64i2,64i3,64i4,65,65i1,65i2,65i3,65i4,65i5,65i6,65i7,65i8,65i9,65i10,65i11,65i12,65i13,66,66i1,67,67i1,67i2,68,68i1,68i2,69,69i1,70,70i1,70i2,70i3,70i4,70i5,70i6,70i7,70i8,70i9,71,71i1,71i2,72,72i1,72i2,73,74,75,76
0,Mammal,eukaryota,Leu,ailMel1_GL192338.1.trna142-LeuCAG,ailMel1,Ailuropoda melanoleuca (panda) (BGI-Shenzhen AilMel 1.0 Dec 2009),9646,Leu,CAG,77.3,124.5,124.5,0,0.463415,0,2,11,7,8,13,True,False,G:C,G,.,U:A,U,.,C:G,C,.,.,.,.,.,.,A:U,A,.,.,.,.,.,.,.,.,.,.,.,G:C,G,.,.,.,.,.,.,.,G:U,G,.,A:U,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,U,U:A:A,U:A,.,.,.,.,G,G:C:G,G:G,.,.,.,G:C,G,G:C:-,G:-,.,.,C:G,C,C:G,C,.,G:A,G,G:A:G,A,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,G,G:C,C,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,-,-,G,G:U,G,G:C,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,U,.,.,.,.,.,.,C,U